In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Churn_MOdelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [4]:
## Extracting independent columns from third to thirteenth
X = df.iloc[:,3:13]

## Extracting dependent column Exited
y = df.iloc[:,13]

In [5]:
# Create dummies varible 
geography = pd.get_dummies(X['Geography'],drop_first=True)

gender = pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
##Now concat the columns in one dataset
X = pd.concat([X,geography,gender],axis=1)

In [7]:
## AFter concat we have to remove these columns
X = X.drop(['Geography','Gender'],axis=1)

In [8]:
## NO splitting my data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
print(X_train.shape)
print(X_test.shape)

(8000, 10)
(2000, 10)


### Now some preprocessing
### Feature scaling

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Let's Create ANN

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import LeakyRelu,PRelu,ELU
from keras.layers import Dropout

In [12]:
## Initialising the ANN
classifier = Sequential()

## Adding the input layer and the first hidden layer

In [13]:
# Weight initilization technique
# 1. he_normal
# 2. he_uniform

classifier.add(Dense(units=10, kernel_initializer = 'he_normal', activation='relu',input_dim=11))
classifier.add(Dropout(0.3))

# Adding the second hidden layer
classifier.add(Dense(units=20, kernel_initializer = 'he_normal', activation='relu'))
classifier.add(Dropout(0.4))

# Adding the third hidden layer
classifier.add(Dense(units=20, kernel_initializer = 'he_normal', activation='relu'))
classifier.add(Dropout(0.2))

# Adding output layer
classifier.add(Dense(units=1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))

# Complete the ANN
classifier.compile(optimizer='adam', loss = 'binary_crossentropy',metrics=['accuracy'])

In [14]:
# Fitting the ANN to the training set
model_history = classifier.fit(X_train,y_train, validation_split=0.33,batch_size = 10, epochs=100)

Epoch 1/100


ValueError: in user code:

    File "C:\python37\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\python37\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\python37\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\python37\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\python37\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\python37\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '

    ValueError: Input 0 of layer "sequential" is inco

In [ ]:
print(model_history.history.keys())

In [ ]:
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
# Predicting the test set result
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

In [ ]:
# Calculating the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [ ]:
# Calculate the accuracy
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,y_test)
score